#### Installation of prereqs

In [ ]:
import torch

!pip install ogb

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

#### Imports

In [ ]:
import random
import numpy as np

import torch
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling, to_networkx
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

import pandas as pd

from helpers import *

#### Functions for train and test

In [ ]:
def train(model, predictor, edge_attr, x, emb_ea, adj_t, split_edge, optimizer, batch_size):
    edge_index = adj_t

    model.train()
    predictor.train()

    pos_train_edge = split_edge['train']['edge'].to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size, shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t, edge_attr, emb_ea)

        # Get positive loss
        edge = pos_train_edge[perm].t()
        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        # Get negative loss (Sample using torch_geometric.utils.negative_sampling)
        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='dense')
        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        # Get total loss and gradients
        loss = pos_loss + neg_loss
        loss.backward()

        # Clip gradient (Limit maximum l2 norm of gradients to 1)
        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples


@torch.no_grad()
def test(model, predictor, edge_attr, x, emb_ea, adj_t, split_edge, evaluator, batch_size):
    model.eval()
    predictor.eval()

    h = model(x, adj_t, edge_attr, emb_ea)

    pos_valid_edge = split_edge['valid']['edge'].to(x.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(x.device)
    pos_test_edge = split_edge['test']['edge'].to(x.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(x.device)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    results = {}
    for K in [20, 50, 100]:
        evaluator.K = K
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (valid_hits, test_hits)

    return results

#### Set up model

In [ ]:
# Model and training parameters
args={
      "device":0, "num_layers":2, "num_samples":5, "node_emb":256, 
      "hidden_channels":256, "dropout":0.3, "batch_size":64 * 1024,
      "lr":0.003, "epochs":200, "log_steps":1, "eval_steps":1, 
      "runs":1
      }

In [ ]:
# Set device
device = f'cuda:{args["device"]}' if torch.cuda.is_available() else 'cpu'

# Load OGB Dataset
dataset = PygLinkPropPredDataset(name='ogbl-ddi')
data = dataset[0]
edge_index = data.edge_index.to(device)
split_edge = dataset.get_edge_split()

# Load model
model = GraphSAGE(args["node_emb"], args["hidden_channels"], args["hidden_channels"],
                  args["num_layers"], args["dropout"]).to(device)
emb = torch.nn.Embedding(data["num_nodes"], args["node_emb"]).to(device)
emb_ea = torch.nn.Embedding(args["num_samples"], args["node_emb"]).to(device)
predictor = LinkPredictor(args["hidden_channels"], args["hidden_channels"], 1,
                          args["num_layers"]+1, args["dropout"]).to(device)

print('Number of parameters:',
      sum(p.numel() for p in list(model.parameters()) +
      list(predictor.parameters()) + list(emb.parameters()) + list(emb_ea.parameters())))

#### Generate SPD features

In [ ]:
# Encode distance information
np.random.seed(0)
nx_graph = to_networkx(data, to_undirected=True)
node_mask = []
for _ in range(args["num_samples"]):
    print(_)
    node_mask.append(np.random.choice(500, size=200, replace=False))
node_mask = np.array(node_mask)
node_subset = np.random.choice(nx_graph.number_of_nodes(), size=500, replace=False) # chose 200 random nodes in our graph
spd = get_spd_matrix(G=nx_graph, S=node_subset, max_spd=5)

# edge_attr: For all edges in the entire graph, collect the distance features for each node in each pair
edge_attr = spd[edge_index, :]

# edge_attr: We now create an edge feature averaging over the node features for each edge (Two nodes per edge)
edge_attr = edge_attr.mean(0)

# edge_attr: For all these edges, we sample K subsets of N of our features
edge_attr = torch.tensor(edge_attr[:, node_mask].mean(2)).to(device)

# edge_attr: Min max normalize
a_max = torch.max(edge_attr, dim=0, keepdim=True)[0]
a_min = torch.min(edge_attr, dim=0, keepdim=True)[0]
edge_attr = (edge_attr - a_min) / (a_max - a_min + 1e-6)

#### Set up for training

In [ ]:
# Evaluator
evaluator = Evaluator(name='ogbl-ddi')
loggers = {
    'Hits@20': Logger(args["runs"], args),
    'Hits@50': Logger(args["runs"], args),
    'Hits@100': Logger(args["runs"], args),
}

In [ ]:
#  this is where the training happens
run = 0
random.seed(run)
torch.manual_seed(run)
torch.nn.init.xavier_uniform_(emb.weight)
torch.nn.init.xavier_uniform_(emb_ea.weight)
model.reset_parameters()
predictor.reset_parameters()
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(emb_ea.parameters()) + list(predictor.parameters()), lr=args["lr"])

# Add to device
edge_index = edge_index.to(device)
emb = emb.to(device)
emb_ea = emb_ea.to(device)
model = model.to(device)
predictor = predictor.to(device)

# Train
for epoch in range(1, 1 + args["epochs"]):
    loss = train(model, predictor, edge_attr, emb.weight, emb_ea.weight, edge_index, split_edge,
                  optimizer, args["batch_size"])
    
    # Test
    if epoch % args["eval_steps"] == 0:
        results = test(model, predictor, edge_attr, emb.weight, emb_ea.weight, edge_index, split_edge,
                        evaluator, args["batch_size"])

        # Logging and prints (Nothing important happens here)
        for key, result in results.items():
            loggers[key].add_result(run, result)
        if epoch % args["log_steps"] == 0:
            for key, result in results.items():
                valid_hits, test_hits = result
                print(key)
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Valid: {100 * valid_hits:.2f}%, '
                      f'Test: {100 * test_hits:.2f}%')
            print('---')
for key in loggers.keys():
    print(key)
    loggers[key].print_statistics(run)


Hits@20
Run: 01, Epoch: 01, Loss: 1.1029, Valid: 0.21%, Test: 0.13%
Hits@50
Run: 01, Epoch: 01, Loss: 1.1029, Valid: 0.34%, Test: 0.35%
Hits@100
Run: 01, Epoch: 01, Loss: 1.1029, Valid: 0.57%, Test: 0.64%
---
Hits@20
Run: 01, Epoch: 02, Loss: 0.8482, Valid: 0.78%, Test: 0.55%
Hits@50
Run: 01, Epoch: 02, Loss: 0.8482, Valid: 1.42%, Test: 1.48%
Hits@100
Run: 01, Epoch: 02, Loss: 0.8482, Valid: 2.49%, Test: 2.23%
---
Hits@20
Run: 01, Epoch: 03, Loss: 0.7193, Valid: 0.71%, Test: 0.80%
Hits@50
Run: 01, Epoch: 03, Loss: 0.7193, Valid: 1.71%, Test: 1.72%
Hits@100
Run: 01, Epoch: 03, Loss: 0.7193, Valid: 3.04%, Test: 2.45%
---
Hits@20
Run: 01, Epoch: 04, Loss: 0.6039, Valid: 6.19%, Test: 3.20%
Hits@50
Run: 01, Epoch: 04, Loss: 0.6039, Valid: 10.78%, Test: 5.88%
Hits@100
Run: 01, Epoch: 04, Loss: 0.6039, Valid: 14.72%, Test: 11.19%
---
Hits@20
Run: 01, Epoch: 05, Loss: 0.5184, Valid: 15.38%, Test: 5.73%
Hits@50
Run: 01, Epoch: 05, Loss: 0.5184, Valid: 19.16%, Test: 10.76%
Hits@100
Run: 01, Epoc

In [ ]:

hits_20 = loggers["Hits@20"].results[0]

df_20 = pd.DataFrame(hits_20)

df_20.to_csv("original_baseline_hits_20.csv")